In [1]:
import re
from functools import partial

## Preparing the List of Prime Gaps

The file `raw_prime_gaps_list.txt` contains the list of known lead prime gaps for all gaps up to $9998$. The data is uniformly stored. A typical row looks like the following, and the meaning of each element of the row is described below.

    6* CFC Glaisher 1877   1.91     2  23
    || |   |        |      |        |  |
    || |   |        |      |        |  ---> first prime in the prime gap
    || |   |        |      |        ------> number of digits of the first prime
    || |   |        |      ---------------> "merit", which is bigger if the gap is bigger wrt prime size
    || |   |        ----------------------> year of discovery
    || |   -------------------------------> discoverer
    || -----------------------------------> C if "conventional gap" (all here are)
    ||                                      F if known first occurrence of gap size, N if not, ? if unknown
    ||                                      C if certified primes, P if probabilistic primes
    |-------------------------------------> * if a "maximal gap", larger than all previous gaps
    --------------------------------------> size of gap
    
Sometimes the first prime is stored not as a number, but as an expression in the form $a \cdot b\# / c - d$, where $b\#$ denotes a *primorial*, or as an expression in the form $2^a + b$. We first read in the list. We extract those rows which are maximal (i.e. has * following the gapsize). Then we keep the gap size and the first prime.

In [2]:
with open("raw_prime_gaps_list.txt", "r") as f:
    lines = f.readlines()
print(lines[3])

     6* CFC Glaisher 1877   1.91     2  23



In [39]:
# First we test the method on just the first 10 prime gaps
gaps = []
gaps_dict = dict()
for line in lines[:10]:
    gap, first_prime = extract_gap_and_prime(line)
    gaps.append(gap)
    gaps_dict[gap] = [first_prime, first_prime + gap]

In [40]:
# These are the gaps
gaps

[1, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [41]:
# Clear prime pair storage
gaps_dict

{1: [2, 3],
 2: [3, 5],
 4: [7, 11],
 6: [23, 29],
 8: [89, 97],
 10: [139, 149],
 12: [199, 211],
 14: [113, 127],
 16: [1831, 1847],
 18: [523, 541]}

In [42]:
# First we test the method on just the first 10 prime gaps
gaps = []
gaps_dict = dict()
for line in lines:
    gap, first_prime = extract_gap_and_prime(line)
    gaps.append(gap)
    gaps_dict[gap] = [first_prime, first_prime + gap]

In [45]:
# Choose random representative to check
gaps_dict[116]

[5845193, 5845309]

In [46]:
def is_maximal_gap(splitlist):
    if "*" in splitlist[0]:
        return True
    return False

def extract_gap_and_prime(line):
    splitlist = line.split(None, 6) # Split on whitespace (None), and make at most 6 splits
                                    # Splitting at most 6 times keeps the expression for the first prime together
    gap_str = splitlist[0]
    first_prime_str = splitlist[6]
    first_prime_str = first_prime_str.rstrip()
    
    #assert gap_str[-1] == "*"
    if gap_str[-1] == "*":
        gap = gap_str[:-1]
    else:
        gap = gap_str
    gap = int(gap)
    
    if "*" in first_prime_str:
        first_prime = extract_primorial(first_prime_str)
    elif "^" in first_prime_str:
        first_prime = extract_two_power_prime(first_prime_str)
    else:
        first_prime = int(first_prime_str)
        
    return gap, first_prime


def memoize(f):
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper


primes_to_541 = primes_first_n(100)
@memoize
def primorial(n):
    return prod([p for p in primes_to_541 if p<= n])


def extract_primorial(expression):
    pattern = re.compile(r"(\w+)\W*\*\W*(\w+)\W*\#\W*/\W*(\w+)\W*-\W*(\w+)")
    m = pattern.match(expression)
    a,b,c,d = map(int, m.groups())
    return a*primorial(b)/c - d


def extract_two_power_prime(expression):
    pattern = re.compile(r"2\^(\w+)\W*\+\W*(\w+)")
    m = pattern.match(expression)
    exp, addend = map(int, m.groups())
    return 2**exp + addend

## Setting up the model functions

We will model the distribution of primes in a few different ways. The nature of the distribution of primes is uncertain. By the prime number theorem, the average prime gap looks like
$$ X, X + \log X.$$
We expect instead that there are many gaps that are much larger. Cramer thought that prime gaps were never larger than
$$ X, X + \log^2 X.$$
It turns out that he wasn't correct, but it may be true that there is a constant $B$ such that gaps are never larger than
$$ X, X + B \log^2 X.$$
It is known that this constant, if it exists, is at least $2e^{-\gamma} \approx 1.1$.
It is also conjectured that no such constant exists, but that for any $A > 2$ there exists a corresponding constant (which I also denote by $B$) such that gaps are never larger than
$$ X, X + B \log^A X.$$

Recall in the background that we conjecture that for each $\alpha$ there exists a constant $C(\alpha)$ such that
$$ p_{n+1}^\alpha - p_n^\alpha \leq C(\alpha).$$

We will model how gaps behave according to these models and compare them to the actual behavior.
To do this, we define the function
$$ f(A, B, \alpha, x) := (X + B \log^A X)^\alpha - X^\alpha.$$
For each fixed $A, B$, and $\alpha$, maximizing $f_{A, B, \alpha}(x)$ gives a largest-possible estimate of the value of $C(\alpha)$. We say largest-possible because this presupposes that there are primes at exactly the maximizing value of $x$, which corresponds to $x$ and $x + B \log^A x$ being primes themselves.

Note that if Cramer's Conjecture were true (for instance), then it may be that the $C(\alpha)$ predictions we get from the Cramer model might be slightly larger than reality, and in general each model errs on the side of overestimating rather than underestimating.

We will set up model functions with values of $(A, B)$ being $(1,1)$ (PNT), $(2,1)$ (Naive Cramer), $(2,2e^{-\gamma})$ (Current Cramer), $(2, 10)$ (Cramer), and $(2.5, 1)$ (Adleman-McCurley).

### Maximizing the model functions

Maximizing $f_{A,B,\alpha}(x)$ is done naively. Compute the derivative and find when it's zero. There is a unique zero greater than $1$, which is our maximum point. A first-order estimation shows that $f_{A,B,\alpha}$ is maximized near $m(A,B,\alpha) = e^{A/(1-\alpha)}$. Note that $B$ does not affect this first-order estimation, so I do not expect the three variants of Cramer to behave significantly differently.

We have that
$$ \begin{align}
f(A, B, \alpha, x) &= (x + B \log^A x)^\alpha - x^\alpha, \\
f'(A, B, \alpha, x) &= \alpha(x + B \log^A x)^{\alpha - 1} \left( 1 + AB \frac{\log^{A-1} x}{x} \right) - \alpha x^{\alpha - 1}.
\end{align}$$

Note that $f' = 0$ (for $x > 1$) is the same as $f'/(\alpha x^{\alpha - 1}) = 0$, which can be rearranged into
$$ g_{A, B, \alpha}(x) := \left( 1 + AB \frac{\log^{A-1} x}{x} \right) - \left( 1 + B \frac{\log^A x}{x} \right)^{1 - \alpha} = 0.$$

This is a pretty well-behaved function, and numerical root solvers should be able to handle it. This is what we implement now.

In [84]:
def F(A, B, alpha, x):
    return (x + B*log(x)**A)**alpha - x**alpha

In [47]:
def g(A, B, alpha, x):
    return (1 + A * B * log(x)**(A-1)/x) - (1 + B*log(x)**A / x)**(1-alpha)

In [75]:
find_root(partial(g,2,1,.931), 2, 10**17)

3875382310049.5977

Actually, this caps out at 17 digit precision and won't suffice for $\alpha > 0.93$. But for really large $x$, the binomial expansion should become more and more accurate, leading to $e^{A/(1-\alpha)}$ being a better and better estimate of the maximizing $x$. For $\alpha > 0.93$, I need to use a different technique. This runs into 64-bit precision arithmetic problems. To solve this, for $\alpha > 0.9$, I will use a homegrown bisection rootfinder in a high precision arithmetic sage environment.

In [140]:
RF = RealField(1000)
def RF_F(A, B, alpha, x):
    return (RF(x) + B*RF(log(x))**A)**alpha - RF(x)**alpha

In [151]:
RF_F(2, 1, .995, start)

94.6462551224227108648753845422473395656417225519337464813725076012845018238854333470590655023933791360594639610137378641151902394969304739020994787415718840404250044482497112695518774671285012294290350100667596286309479025540713352067429287243761089151330174642895411023487669065453967075785313285289

In [152]:
def RF_g(A, B, alpha, x):
    return (RF(1) + A * B * RF(log(x))**(A-1)/RF(x)) - (RF(1) + B*RF(log(x))**A / RF(x))**(1-alpha)

In [153]:
def RF_gsim(x):
    return RF_g(2,1,0.8,x)

In [154]:
start = exp(2/.2)

In [155]:
RF_gsim(start)

1.64444546432196045785420540155893726063200274572267336535350345517820986134145980935118162918424805451405780491393962735635632068351784816308968439342270391699018760425327496611550914706761553155070977740717893810316453496034348416368377098158251454957552350910050505616367388235938291760692342296640e-6

In [159]:
find_root(RF_gsim, 2, 10**17)

22423.973931885273

In [211]:
# Note that this requires that func(2*start) < 0 to work properly
# In our case, this means that the `start` guess can't be too bad
def bisection_rootfinding(func, start, iterations=1, goal=10**(-7)):
    if start*2**(-iterations) < goal:
        return start
    if func(start) > 0:
        return bisection_rootfinding(func, start + start * 2**(-iterations), iterations=iterations+1, goal=goal)
    return bisection_rootfinding(func, start - start * 2**(-iterations), iterations=iterations+1, goal=goal)

In [161]:
bisection_rootfinding(RF_gsim, start)

22423.9739319633

Notice in particular that this is essentially the same as scipy's find_root result, so my method seems to work pretty well.

Now let's try it on something larger.

In [202]:
def RF_gsim2(x):
    return RF_g(2,1,0.95,x)

In [203]:
start = RF(exp(2/.05))

In [204]:
find_root(RF_gsim2, 2, 10**16)

9999999710581220.0

In [205]:
bisection_rootfinding(RF_gsim2, start)

2.35385266837042022843327073581445209550189717806195566505913760542398087081961498860906821406821821384500066808118082097402013219336066587283757576744855862678903728952660301215698355426390611932122470916392998635276106218830903304888702478640727370013260828376486514621410432957225533987162136976562e17

In [206]:
fr = find_root(RF_gsim2, 2, 10**16)
br = bisection_rootfinding(RF_gsim2, start)

In [207]:
br/fr

23.538527364953389

In [209]:
RF_F(2,1,0.95,br)

205.709630519650881691924521792765193253506704662129143200009919811578164796694847391279127980866813238127336707532494092877547455355110417039290138055484169795377093169784536561532277570158710114624712809121313730522096004967342309066003863302396648990201591070015928728914879882408923972678419871216

In [210]:
RF_F(2,1,0.95,fr)

204.359554823890368403363980346311818555686040610577329335810851723660578683578255065736205720488148424074134614592141667900060298707050938806333994911863866233402912888249520944989763989818618341662786563588774804722056915716107185579765431262102230328894524070859044620196971152617523013227880281789

This is interesting --- there is a region in which find_root fails due to precision reasons, but not in an obviously detectable way. But as this clearly shows, my bisection method suffices.

## Data Creation

The plan is to loop through $\alpha$ from $0.005$ to $0.995$, compute everything, and compare everything. But for many initial $\alpha$, everything is boring. Let's first find exactly when $\alpha$ is boring.

In [300]:
def estimate_c_alpha(alpha):
    MAX = 0
    MAXIMIZING_PRIME = 0
    MAXIMIZING_GAP = 0
    for gap in gaps:  # Note: It is possible to optimize this selection based on vals for other alpha
        a,b = gaps_dict[gap]
        diff = RF(b)**alpha - RF(a)**alpha
        if diff > MAX:
            MAX = diff
            MAXIMIZING_PRIME = a
            MAXIMIZING_GAP = gap
    return MAX, MAXIMIZING_GAP, MAXIMIZING_PRIME

In [301]:
for a in range(20):
    alpha = 0.05 + 0.04 * a
    print alpha, estimate_c_alpha(alpha)

0.0500000000000000 (0.0273310781848302800658590880398522815347590177971289917771832998742533538405129347101901809408899390154774514006487716615848197146224848388750750143388527967917247425289525849885984701315952214994461518869467334113477417254612430110708981668255496105519249618121865605658797754277991125986546424926508, 2, 3)
0.0900000000000000 (0.0519370809347252520685872649136154113889575896783660909311486774384967763294410100201853785718930335741505701554636256358169356949220163741754562328432777159041140235202911046252875832381116056987398716939257761619006205913299310880723598291675100978615964192750372577866405938486631637439508364320858, 2, 3)
0.130000000000000 (0.0792030225749460145408332846210646211349709553727122695589775991379642815865105358693876664136993146934495792581811221651860043234387972830839584009007088431244195480057541946715422880580136690530288636745487275691029720040890241179152279971140525502557180885918952288021915731111539843731655078009650, 2, 3)
0.170000

It appears that starting at $\alpha = 0.005$ will be fine, though not a lot happens until $\alpha \approx 0.5$. The semiimportant data there is really just the value of $C(\alpha)$.

Let us now set up the data structures to store the data. These will be a set of dictionaries, contained in a "model" dictionary.

In [316]:
experimental_model = dict()
PNT_model = dict()
naive_cramer_model = dict()
current_cramer_model = dict()
weak_cramer_model = dict()
AM_model = dict()

models = dict()
models["experimental"] = experimental_model
models["PNT"] = PNT_model
models["naive"] = naive_cramer_model
models["current"] = current_cramer_model
models["weak"] = weak_cramer_model
models["AM"] = AM_model

In each model dictionary, the keys will be values of $\alpha$ from $0.005$ to $0.995$, and the values will be `(C(alpha), gap, lower_prime)` tuples. That's that.

I'll store the data as pickled dictionaries, and as json dumps (for external portability).

In [317]:
RF = RealField(1000)
def RF_F(A, B, alpha, x):
    return (RF(x) + B*RF(log(x))**A)**alpha - RF(x)**alpha

def RF_g(A, B, alpha, x):
    return (RF(1) + A * B * RF(log(x))**(A-1)/RF(x)) - (RF(1) + B*RF(log(x))**A / RF(x))**(1-alpha)


def maximize_model(A, B, alpha):
    fun = lambda x: RF_F(A, B, alpha, x)
    deriv = lambda x: RF_g(A, B, alpha, x)
    estimate_lower_prime = 2*RF(exp(A/(1 - alpha)))
    lower_prime = bisection_rootfinding(deriv, start)
    Calpha = fun(lower_prime)
    gap = B*log(lower_prime)**A
    return (Calpha, gap, lower_prime)

In [318]:
def make_experimental_data(alpha):
    Calpha, gap, lower_prime = estimate_c_alpha(alpha)
    return (Calpha, gap, lower_prime)

def make_model_data(A, B, alpha):
    return maximize_model(A,B,alpha)

def make_PNT_model_data(alpha):
    return make_model_data(1,1,alpha)

def make_naive_model_data(alpha):
    return make_model_data(2,1,alpha)

def make_current_model_data(alpha):
    return make_model_data(2,1.1229189671337703396, alpha)

def make_weak_model_data(alpha):
    return make_model_data(2, 10, alpha)

def make_AM_model_data(alpha):
    return make_model_data(2.5, 1, alpha)

In [319]:
for a in range(1000):
    alpha = 0.000 + a/1000
    models["experimental"][alpha] = make_experimental_data(alpha)
    models["PNT"][alpha] = make_PNT_model_data(alpha)
    models["naive"][alpha] = make_naive_model_data(alpha)
    models["current"][alpha] = make_current_model_data(alpha)
    models["weak"][alpha] = make_weak_model_data(alpha)
    models["AM"][alpha] = make_AM_model_data(alpha)
    if a%50 == 0:
        print a

0
50
100


KeyboardInterrupt: 

In [311]:
#t = models.keys()[0]
#for k in models[t].keys():
#    print k
#    for key in models.keys():
#        print key, "{}".format(float(models[key][k][0]))
#    print

0.875000000000000
weak 103.439058259
naive 10.3439058259
AM 64.3968893029
PNT 0.266884834048
current 11.6153678894
experimental 22.355658536

0.985000000000000
weak 8912.19576218
naive 891.219576218
AM 5697.45715275
PNT 21.8068038374
current 1000.76953125
experimental 832.776922321

0.845000000000000
weak 31.229314907
naive 3.1229314907
AM 19.4420827966
PNT 0.0805752746273
current 3.50679779053
experimental 13.1854821199

0.955000000000000
weak 2535.58416111
naive 253.558416111
AM 1620.96777277
PNT 6.47005390334
current 284.7265625
experimental 223.130084973

0.815000000000000
weak 9.41656698088
naive 0.941656698088
AM 5.86236603164
PNT 0.0242958410324
current 1.05740404129
experimental 8.63920435249

0.925000000000000
weak 759.339803845
naive 75.9339803845
AM 472.733628038
PNT 1.9591852531
current 85.2677001953
experimental 75.480606344

0.895000000000000
weak 229.692445411
naive 22.9692445411
AM 142.997038351
PNT 0.592633297398
current 25.7925872803
experimental 33.0294996982

0.8650

In [315]:
estimate_c_alpha(0.999)

(7274.01029667115663804696139950196546879133311653806677755178641413853004747248665248732733004153158249664271458453082621203090220754595578307388531950622042827007945883013060182605287118742812745971870821132458880856535628367573150275722978263740127429950379663513796961444341028077742437063723642042,
 9738,
 1825353380273296795064168516303102324850159011450737221110852890827600657469117119943865647903448359184405879506116913454583803)